In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width: 90% !important; }</style>"))

# "How's that movie?" -  Collaborative filtering with FastAI

*Build a state-of-the-art movie recommendation system with just 10 lines of code*

<img src="https://cdn-images-1.medium.com/max/1200/1*UCnwjtYoYoJtWM7DxzRT0A.jpeg" style="width:480px;"/>

Recommender systems are at the core of pretty much every online service we interact with. Social networking sites like Facebook, Twitter and Instagram recommend posts you might like, or people you might know. Video streaming services like YouTube and Netflix recommend videos, movies or TV shows you might like. Online shopping sites like Amazon recommend products you might want to buy. 

**Collaborative** filtering is perhaps the most common technique used by recommender systems.

> *Collaborative filtering is a method of making predictions about the interests of a user by collecting preferences from many users. The underlying assumption is that if a person A has the same opinion as a person B on an issue, A is more likely to have B's opinion on a different issue than that of a randomly chosen person.* - [Wikipedia](https://medium.com/r/?url=https%3A%2F%2Fen.wikipedia.org%2Fwiki%2FCollaborative_filtering)

The [librec](https://medium.com/r/?url=https%3A%2F%2Fwww.librec.net%2F) Java library provides [over 70 different algorithms](https://medium.com/r/?url=https%3A%2F%2Fwww.librec.net%2Fdokuwiki%2Fdoku.php%3Fid%3DAlgorithmList%23recommender_algorithm_list) for collaborative filtering. In this post however, we'll implement a relatively new technique called *neural collaborative filtering*.

## The MovieLens 100K Dataset

The [MovieLens 100K dataset](https://grouplens.org/datasets/movielens/) is a collection of movie ratings by 943 users on 1682 movies. There are 100,000 ratings in total, since not every user has seen and rated every movie. Here are some sample ratings from the dataset:

<img src="https://cdn-images-1.medium.com/max/800/1*OZLI7a0ueujHzj5NG3oRlQ.png" style="width:360px" />

Every user is given a unique numeric ID (ranging from 1 to 943), and each movie is given a unique numeric ID too (ranging from 1 to 1682). User's ratings for movies are integers ranging from 1 to 5, with 5 being the highest.

Our objective here is to build a *model* that can predict how a user would rate a movie they haven't already seen, by looking at the movie ratings of other users with similar tastes.

## System Setup

If you want to follow along and run the code as you read, you can clone this notebook, install the required dependencies using [conda](https://conda.io), and start Jupyter by running the following commands on the terminal:

```
pip install jovian --upgrade     # Install the jovian library 
jovian clone 5bc23520933b4cc187cfe18e5dd7e2ed # Download notebook
cd movielens-fastai              # Enter the created directory 
jovian install                   # Install the dependencies
conda activate movielens-fastai  # Activate virtual environment
jupyter notebook                 # Start Jupyter
```

Make sure you have [conda](https://conda.io) installed before running the above commands. You can also click on the **"Run on Binder"** button at the top to start a Jupyter notebook server hosted on mybinder.org  instantly.

## Preparing the Data

You can download the MovieLens 100K dataset [from this link](https://medium.com/r/?url=http%3A%2F%2Ffiles.grouplens.org%2Fdatasets%2Fmovielens%2Fml-100k.zip). Once downloaded, unzip and extract the data into a directory ml-100k next to the Jupyter notebook. As described in the [README](https://medium.com/r/?url=http%3A%2F%2Ffiles.grouplens.org%2Fdatasets%2Fmovielens%2Fml-100k-README.txt), the file u.data contains the list of ratings.

*On Linux and Mac, you can simply run the follwing cell to download and extract the data:*

In [20]:
dataset_url = "http://files.grouplens.org/datasets/movielens/ml-100k.zip"

In [21]:
from urllib.request import urlretrieve
print("Processing...")
urlretrieve(dataset_url, "ml-100k.zip")
from zipfile import ZipFile
with ZipFile ('ml-100k.zip') as zip:
    zip.printdir()
    zip.extractall()
    print("Done!")

Processing...
File Name                                             Modified             Size
ml-100k/                                       2016-01-29 14:26:28            0
ml-100k/allbut.pl                              2000-07-19 16:09:28          716
ml-100k/mku.sh                                 2000-07-19 16:09:28          643
ml-100k/README                                 2016-01-29 14:26:28         6750
ml-100k/u.data                                 2000-07-19 16:09:30      1979173
ml-100k/u.genre                                2000-07-19 16:09:30          202
ml-100k/u.info                                 2000-07-19 16:09:30           36
ml-100k/u.item                                 2000-07-19 16:09:30       236344
ml-100k/u.occupation                           2000-07-19 16:09:30          193
ml-100k/u.user                                 2000-07-19 16:09:30        22628
ml-100k/u1.base                                2001-03-08 12:33:08      1586544
ml-100k/u1.test           

We begin the importing the required modules from Pandas and FastAI.

In [13]:
!pip install fastai==1.0.50 --quiet

You should consider upgrading via the 'c:\program files (x86)\microsoft visual studio\shared\python37_64\python.exe -m pip install --upgrade pip' command.


In [24]:
import pandas as pd
from fastai.collab import CollabDataBunch, collab_learner

We can now read the data from the CSV file `u.data` into a Pandas [data frame](https://medium.com/r/?url=https%3A%2F%2Fpandas.pydata.org%2Fpandas-docs%2Fstable%2Freference%2Fapi%2Fpandas.DataFrame.html), and create a FastAI [data bunch](https://medium.com/r/?url=https%3A%2F%2Fdocs.fast.ai%2Fbasic_data.html%23Get-your-data-ready-for-training) which:

1. Converts the Pandas data frame into tensors
2. It splits the data into a training set and a validation set
3. Creates data loaders to access the data in batches
4. Checks if a GPU is available, and moves the data to the GPU

In [25]:
cols = ['User ID','Movie ID','Rating','Timestamp']
ratings_df = pd.read_csv('ml-100k/u.data', delimiter='\t', 
                         header=None, names=cols)
ratings_df.sample(5)

,User ID,Movie ID,Rating,Timestamp
3905,130,550,5,878537602
9232,5,388,2,879198898
34521,109,55,2,880572756
76361,788,679,2,880871057
21727,13,478,4,884538571


In [32]:
data = CollabDataBunch.from_df(ratings_df, valid_pct=0.1)
data

TabularDataBunch;

Train: LabelList (90000 items)
x: CollabList
User ID 196; Movie ID 242; ,User ID 186; Movie ID 302; ,User ID 22; Movie ID 377; ,User ID 244; Movie ID 51; ,User ID 298; Movie ID 474; 
y: FloatList
3.0,3.0,1.0,2.0,4.0
Path: .;

Valid: LabelList (10000 items)
x: CollabList
User ID 804; Movie ID 143; ,User ID 410; Movie ID 689; ,User ID 308; Movie ID 433; ,User ID 933; Movie ID 64; ,User ID 181; Movie ID 620; 
y: FloatList
3.0,2.0,5.0,5.0,2.0
Path: .;

Test: None

## Collaborative filtering model

The model itself is quite simple. We represent each user `u` and each movie `m` by vector of a predefined length `n`. The rating for the movie `m` by the user `u`, as predicted by the model is simply the dot product of the two vectors.

<img src="https://cdn-images-1.medium.com/max/800/1*RuAjbXDwvTAv74NtPS-FbQ.png" style="width:360px"/>

Here's a small subset of the users and movies, represented by randomly chosen vectors of length 5, and the predicted ratings:

<img src="https://cdn-images-1.medium.com/max/1600/1*y9H2v3PsytrCajWSNJJDsA.png" style="width:480px"/>

Since the vectors are chosen randomly, it's quite unlikely that the ratings predicted by the model match the actual ratings. Our objective, while training the model, is to gradually adjust the elements inside the user & movie vectors so that predicted ratings get closer to the actual ratings.

We can use the `collab_learner` method from `fastai` to create a neural collaborative filtering model.

In [33]:
learn = collab_learner(data, n_factors=40, y_range=[0,5.5], wd=.1)

The actual model created here contains 2 important enhancements on the simpler version described earlier:

First, apart from the vectors for users and movies, it also add bias terms to account for outliers, since some users tend to always rate movies very high or very low, and some movies tend to be universally acclaimed or disliked.

<img src="https://cdn-images-1.medium.com/max/800/1*vF7oEwpuKfXP8_CYK_KgdQ.png" style="width:360px" />

Second, it applies the Sigmoid activation function to the above output, and scales it so that the result always lies in the given `y_range`, which is 0 to 5.5 in this case.

<img src="https://cdn-images-1.medium.com/max/800/1*NloTSPs4pCMYDZ8G1kxhoQ.png" style="width:420px"/>

In [36]:
ratings_df

,User ID,Movie ID,Rating,Timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


## Training the model


<img src="https://www.deepnetts.com/blog/wp-content/uploads/2019/02/SupervisedLearning.png" width="640">

The learner uses the [*mean squared error*](https://en.wikipedia.org/wiki/Mean_squared_error) loss function to evaluate the predictions of the model, and the [Adam optimizer](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/) to adjust the parameters (vectors and biases) using [gradient descent](https://www.youtube.com/watch?v=sDv4f4s2SB8). Before we train the model, we use the [learning rate finder](https://docs.fast.ai/callbacks.lr_finder.html) to select a good learning for the optimizer.

In [37]:
learn.lr_find()
learn.recorder.plot(skip_end=15)

epoch,train_loss,valid_loss,time


LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


BrokenPipeError: [Errno 32] Broken pipe

Upon inspection of the graph, we can see that the decrease in loss starts to decrease when the learning rate is around `0.01`. We can choose this as our learning rate, and train for 5 epochs, while annealing the learning rate using the [1-cycle policy](https://sgugger.github.io/the-1cycle-policy.html), which leads to faster convergence.

In [8]:
learn.fit_one_cycle(5, 0.01)

epoch,train_loss,valid_loss,time
0,0.935871,0.950807,00:11
1,0.879205,0.900394,00:08
2,0.798324,0.842735,00:08
3,0.671610,0.809347,00:08
4,0.469696,0.810611,00:08


In just 30 seconds, the mean squared error has come down to around 0.80, which is quite close to the state of the art (as compared with [these benchmarks](https://medium.com/r/?url=https%3A%2F%2Fwww.librec.net%2Frelease%2Fv1.3%2Fexample.html)). And it only took us 8 lines of code to load the data and train the model!

## Looking at some predictions

While it's great to see the loss go down, let's look at some actual predictions of the model.

In [9]:
(users, items), ratings = next(iter(data.valid_dl))
preds = learn.model(users, items)
print('Real\tPred\tDifference')
for p in list(zip(ratings, preds))[:16]:
    print('{}\t{:.1f}\t{:.1f}'.format(p[0],p[1],p[1]-p[0]))

Real	Pred	Difference
4.0	3.7	-0.3
4.0	3.1	-0.9
4.0	3.5	-0.5
3.0	2.9	-0.1
5.0	3.6	-1.4
3.0	3.9	0.9
4.0	3.9	-0.1
4.0	4.5	0.5
4.0	3.4	-0.6
3.0	3.0	-0.0
3.0	2.2	-0.8
3.0	3.5	0.5
5.0	3.9	-1.1
4.0	3.2	-0.8
3.0	3.0	0.0
3.0	3.1	0.1


Indeed, the predictions are quite close to the actual ratings. We can now use this model to predict how users would rate movies they haven't seen, and recommend movies that have a high predicted rating.

<img src="https://cdn-images-1.medium.com/max/800/1*YXlBhnk7cQw5cjbW9qBm0A.png" style="max-width:480px" />

## Save and commit

As a final step, we can save and commit our work using the `jovian` library.

In [10]:
!pip install jovian --upgrade -q

In [11]:
import jovian

In [12]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Updating notebook "hasnainmehmood3435/movielens-fastai" on https://jovian.ai
[jovian] Committed successfully! https://jovian.ai/hasnainmehmood3435/movielens-fastai


'https://jovian.ai/hasnainmehmood3435/movielens-fastai'

Jovian uploads the notebook to [jvn.io](https://jvn.io), captures the Python environment and creates a sharable link for the notebook. You can use this link to share your work and let anyone reproduce it easily with the `jovian clone` command. Jovian also includes a powerful commenting interface, so you (and others) can discuss & comment on specific parts of your notebook.

## Further Reading

In a future post, we'll dive deeper and see how `DataBunch` and `collab_learner` are actually implemented, using PyTorch. We'll also explore how we can interpret the vectors and biases learned by the model, and see some interesting results.

In the meantime, following are some resources if you'd like to dive deeper into the topic:

* [Lesson 4](https://course.fast.ai/videos/?lesson=4) of FastAI's "Practical Deep Learning for Coders" course

* [Paper](https://arxiv.org/abs/1708.05031) introducing neural collaborative filtering

* [PyTorch: Zero to GANs](https://medium.com/jovian-io/pytorch-basics-tensors-and-gradients-eb2f6e8a6eee)  - tutorial series covering the basics of PyTorch and neural networks